In [47]:
import requests
start="12 яеваря 2021"
start = datetime.strptime()
# Выполнение запроса к API Московской биржи
j = requests.get('http://iss.moex.com/iss/engines/stock/markets/shares/securities/MOEX/candles.json?from={start}}&till={end}&interval=24').json()



TypeError: strptime() takes exactly 2 arguments (0 given)

In [104]:
from datetime import datetime, timedelta
import re
import traceback
import pandas as pd
ticker = "MOEX"

date="6 января в 15:31"

pattern_previous_years = r'^([1-9]|[12][0-9]|3[01])\s+(января|февраля|марта|апреля|мая|июня|июля|августа|сентября|октября|ноября|декабря)\s+(\d{4})$'
pattern_this_year = r'^([1-9]|[12][0-9]|3[01])\s+(января|февраля|марта|апреля|мая|июня|июля|августа|сентября|октября|ноября|декабря)\s+в\s+([01][0-9]|2[0-3]):([0-5][0-9])$'

class CustomError(Exception):
    pass

try:     
    one_day=timedelta(days=1)   
    months = {
            "января": "01",
            "февраля": "02",
            "марта": "03",
            "апреля": "04",
            "мая": "05",
            "июня": "06",
            "июля": "07",
            "августа": "08",
            "сентября": "09",
            "октября": "10",
            "ноября": "11",
            "декабря": "12"
        }

    date = date.split()
    now=datetime.now()
    if date[0]=="Сегодня":
        pass
    elif date[0]=="Вчера":
        date_start=now-timedelta(days=1)
    elif re.match(pattern_previous_years," ".join(date)) is not None:   
        date[1]=months[date[1]]
        date=' '.join(date)
        date_start = datetime.strptime(date, '%d %m %Y')
    elif re.match(pattern_this_year," ".join(date)) is not None:
        date[1]=months[date[1]]
        date.insert(2,"2024")
        date=' '.join(date)  
        date_start = datetime.strptime(date, '%d %m %Y в %H:%M')
    else:
        raise CustomError("Нет паттерна для такого формата даты")
    
    date_end = date_start+one_day

    j = requests.get(f'http://iss.moex.com/iss/engines/stock/markets/shares/securities/{ticker}/candles.json?from={date_start.date()}&till={date_end.date()}&interval=24').json()

    def fix_holidays(j, date_start, date_end):
        if (len(j["candles"]["data"]))==2:
            return j

        elif (len(j["candles"]["data"]))==1:
            while (len(j["candles"]["data"]))!=2:
                if date_start.date()==datetime.strptime(j["candles"]["data"][0][6],"%Y-%m-%d %H:%M:%S").date():
                    date_end=date_end+one_day
                elif date_end.date()==datetime.strptime(j["candles"]["data"][0][6],"%Y-%m-%d %H:%M:%S").date():
                    date_start=date_start-one_day                                   
                else: 
                    raise CustomError("Какая то хрень с выявлением выходных")
                j = requests.get(f'http://iss.moex.com/iss/engines/stock/markets/shares/securities/{ticker}/candles.json?from={date_start.date()}&till={date_end.date()}&interval=24').json()
            return j
        
        elif (len(j["candles"]["data"]))==0:
            date_end=date_end+one_day
            date_start=date_start-one_day 
            j = requests.get(f'http://iss.moex.com/iss/engines/stock/markets/shares/securities/{ticker}/candles.json?from={date_start.date()}&till={date_end.date()}&interval=24').json()
            return fix_holidays(j, date_start, date_end)
    j=fix_holidays(j, date_start, date_end)
    data = [{k: r[i] for i, k in enumerate(j['candles']['columns'])} for r in j['candles']['data']]

    # Вывод данных
    print((data[1]["open"]-data[0]["open"])/data[0]['open'])

except Exception as e:
    print(f"Ошибка: {e}")
    traceback.print_exc()

-0.0054599773359429995


In [90]:
import pandas as pd
j = requests.get(f'http://iss.moex.com/iss/engines/stock/markets/shares/securities/MOEX/candles.json?from=2019-12-30&till=2020-01-01&interval=24').json()
data=pd.DataFrame(columns=j["candles"]["columns"], data=j["candles"]["data"])
data.head()

,open,close,high,low,value,volume,begin,end
0,107.2,107.75,108.2,106.94,430513601.2,3991460,2019-12-30 00:00:00,2019-12-30 23:59:59


In [96]:
j = requests.get(f'http://iss.moex.com/iss/engines/stock/markets/shares/securities/MOEX/candles.json?from=2019-12-30&till=2020-01-01&interval=24').json()
print(j["candles"]["data"][0][6])

2019-12-30 00:00:00
